## User Story: Save openEO result to Workspace

Mia is a data scientist, she wants to have access to the user workspace solution to store user results produced with the openEO Platform service offering.

In order to save results from openEO jobs to workspaces, make sure that either your external workspace is registered with the openEO backend (see External Workspace Registration Notebook) or you have provisioned a local workspace.

Then all you need to do, is add the export_to_workspace process after your save_result process in the process graph

arguments: {"workspace": WORKSPACE_NAME}

In [ ]:
import openeo

# Set This variable to your workspace's name.

WORKSPACE_NAME = ""

# Connect to the openEO backend and authenticate with EGI Check-In

connection = openeo.connect("https://openeo.eodc.eu/openeo/1.1.0")
connection = connection.authenticate_oidc(provider_id="egi")

## Using a process graph

Here all we do is set the workspace argument in the process to whatever our workspace might be called.

The rest of the process just loads and saves data, we need to run save_result first, as otherwise there won't be any data to export.

In [ ]:
process_graph = {
  "categories": [],
  "deprecated": False,
  "experimental": False,
  "process_graph": {
    "load1": {
      "process_id": "load_collection",
      "arguments": {
        "id": "boa_sentinel_2",
        "spatial_extent": {
          "west": 16.156771491786476,
          "east": 16.59018048465475,
          "south": 48.08419286799747,
          "north": 48.34670064966687
        },
        "temporal_extent": [
          "2019-01-01T00:00:00Z",
          "2019-01-31T00:00:00Z"
        ],
        "bands": [
          "B02"
        ]
      }
    },
    "save2": {
      "process_id": "save_result",
      "arguments": {
        "data": {
          "from_node": "load1"
        },
        "options": {},
        "format": "GTIFF"
      },
      "result": True
    },
    "export1": {
      "process_id": "export_to_workspace",
      "arguments": {
        "data": {
          "from_node": "save2"
        },
        "workspace": WORKSPACE_NAME,
        "path": ""
      }
    }
  }
}

In [ ]:
job = connection.create_job(process_graph=process_graph, title="save-to-workspace-job")
job.start_job()

In [ ]:
job # Execute this to check on your jobs progress

After the job has finished you can check your workspace to access your results!